In [2]:
import numpy as np
import cv2
import sys
import time
from pixellib.semantic import semantice_segmentation
from pixellib.instance import instance_segmentation
import matplotlib.pyplot as plt

### 9.3.5 프로그래밍 실습: YOLO v3으로 물체 검출

In [8]:
def construct_yolo_v3():
    f = open("./saved_model/coco_names.txt", 'r')
    class_names = [line.strip() for line in f.readlines()]
    
    model =cv2.dnn.readNet("./saved_model/yolov3.weights", "./saved_model/yolov3.cfg")
    layer_names = model.getLayerNames()
    out_layers = [layer_names[i-1] for i in model.getUnconnectedOutLayers()]
    
    return model, out_layers, class_names


def yolo_detect(img, yolo_model, out_layers):
    height, width = img.shape[0], img.shape[1]
    test_img = cv2.dnn.blobFromImage(img, 1.0/256, (448, 448), (0, 0, 0), swapRB=True)
    yolo_model.setInput(test_img)
    output3 = yolo_model.forward(out_layers)
    
    box, conf, id = [], [], []
    for output in output3:
        for vec85 in output:
            scores = vec85[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                centerx, centery = int(vec85[0]*width), int(vec85[1]*height)
                w, h = int(vec85[2]*width), int(vec85[3]*height)
                x, y = int(centerx - w/2), int(centery - h/2)
                box.append([x, y, x+w, y+h])
                conf.append(float(confidence))
                id.append(class_id)
    ind = cv2.dnn.NMSBoxes(box, conf, 0.5, 0.4)
    objects = [box[i] + [conf[i]] + [id[i]] for i in range(len(box)) if i in ind]
    return objects


model, out_layers, class_names = construct_yolo_v3()
colors = np.random.uniform(0, 255, size=(len(class_names), 3))

img = cv2.imread("./imgs/soccer.jpg")

res = yolo_detect(img, model, out_layers)

for i in range(len(res)):
    x1, y1, x2, y2, confidence, id = res[i]
    text = str(class_names[id]) + "%.3f" %confidence
    cv2.rectangle(img, (x1, y1), (x2, y2), colors[id], 2)
    cv2.putText(img, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
cv2.imshow("Object detection by YOLO v.3", img)
cv2.waitKey()
cv2.destroyAllWindows()

In [14]:
cap = cv2.VideoCapture("video/boy.mov")

start = time.time()
n_frame = 0
while True:
    ret, frame = cap.read()
    
    res = yolo_detect(frame, model, out_layers)
    
    for i in range(len(res)):
        x1, y1, x2, y2, confidence, id = res[i]
        text = str(class_names[id]) + "%.3f"%confidence
        cv2.rectangle(frame, (x1, y1), (x2, y2), colors[id], 2)
        cv2.putText(frame, text, (x1, y1+30), cv2.FONT_HERSHEY_PLAIN, 1.5, colors[id], 2)
    
    cv2.imshow("object detection from video by YOLO v.3", frame)
    n_frame += 1
    
    key = cv2.waitKey(1)
    if key == ord('q'): break

end = time.time()
print("처리한 프레임수 =", n_frame, ", 경과시간=", end-start, "\n초당 프레임 수=", n_frame/(end-start))

cap.release()
cv2.destroyAllWindows()

처리한 프레임수 = 23 , 경과시간= 6.642044544219971 
초당 프레임 수= 3.4627891828902326


### 9.4.5 pixellib 라이브러리를 이용한 프로그래밍 실습

In [ ]:
# 정지 이미지 semantic segmentation
seg = semantic_segmentation()
seg.load_ade20k_model("./saved_model/deeplabv3_xception65_ade20k.h5")


img_fname = "./imgs/busy_street.jpg"
seg.segmentAsAde20k(img_fname, output_image_name="image_new.jpg")
info1, img_segmented1 = seg.segmentAsAde20k(img_fname)
info2, img_segmented2 = seg.segmentAsAde20k(img_fname, overlay=True)


plt.subplot(1, 3, 1)
plt.imshow(cv2.imread(img_fname)[..., ::-1])
plt.subplot(1, 3, 2)
plt.imshow(img_segmented1[..., ::-1])
plt.subplot(1, 3, 3)
plt.imshow(img_segmented2[..., ::-1])

In [ ]:
# 비디오 semantic segmentation
cap = cv2.VideoCapture("./")

seg_video = semantic_segmentation()
seg_video.load_ade20k_model("./saved_model/deeplabv3_xception65_ade20k.h5")

seg_video.process_camera_ade20k(cap, overlay=True, frames_per_second=2, output_vidieo_name="output_video.mp4",
                               show_frames=True, frame_name="Pixellib")

cap.release()
cv2.destroyAllWindows()

In [ ]:
# 정지 이미지 instance segmentation
seg = instance_segmentation()
seg.load_model("mask_rcnn_coco.h5")

img_fname = "./imgs/busy_street.jpg"
infro, img_segmented = seg.segmentImage(img_frname, show_bboxe=True)

plt.imshow(img_segmented[... ::-1])

In [ ]:
# 비디오 instance segmentation
cap = cv2.VideoCapture("./")

seg_video = instance_segmentation()
seg_video.load_model("mask_rcnn_coco.h5")

target_class = seg_video.select_target_classese(person=True)
seg_video.process_camera(cap, segment_target_classes=target_calss, frames_per_second=2,
                        show_frames=True, frame_name="Pixellib", show_bboxes=True)

cap.release()
cv2.destroyAllWindows()